In [1]:
!pip install praw
!pip install nltk
!pip install sqlalchemy
!pip install psycopg2-binary

In [2]:
from configparser import ConfigParser
import praw
from praw.models import MoreComments
import pandas as pd
import matplotlib.pyplot as plt
import json
import plotly
import plotly.express as px
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import requests
from flask import Flask, render_template
from datetime import datetime, timedelta 
from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [69]:



reddit = praw.Reddit(
      client_id= '',
      client_secret = '',
      password = '',
      user_agent = '',
      username = '',
      check_for_async=False
  )

  #coinmarketcap API call - limit to top 150 coins
url='https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
for start in range(1, 100, 1000):
    params = {
        'start': start,
        'limit': 150,
            }
    r = requests.get(url, params=params)
    data = r.json()

ticker = [(crypto['symbol'].lower()) for crypto in (data['data'])]
slug = [(crypto['name'].lower()) for crypto in (data['data'])]

  #subreddit to set parameters for title, author, body, score
def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for topics in hot_topic: 
    topic_list = topics.comments.list()
    for comments in topic_list:
      if isinstance(comments, MoreComments):
        continue
      temp_df = pd.DataFrame({'title': [topics.title],
                        'author': [comments.author],
                        'id': [comments.id],
                        'comments': [comments.body],
                        'created_utc': [comments.created_utc],
                        'upvote_score': [comments.score]})
      df = pd.concat([df, temp_df], ignore_index=True)

  return df

wall = reddit_scraper(reddit, 'CryptoCurrency', 49)

tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

#text cleaning; tokenization, stop words, lemmatization
def text_cleaner(df):
    cleaned_text = []
    for text in df:
        tokenize_words = tokenizer.tokenize(text)      
        stop_word_filter = [token.lower() for token in tokenize_words if token.lower() not in stop_words and not token.isdigit()]
        lemmitize_words = ([lemmatizer.lemmatize(token) for token in stop_word_filter])
        cleaned_text.extend(lemmitize_words)
    
    return cleaned_text


                    
def sentiment_analysis(df):
  return text_cleaner(df)

#convert utc time to datetime
def get_date(submission):
	time = submission.created
	return datetime.datetime.fromtimestamp(time)

#login to postgres
def postgres_login(user, password, host, port, db, table_name):   
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
    wall.to_sql(name=table_name, con=engine, if_exists='replace')
    
    
sia = SIA()

ps = []
for comment in wall.comments:
    #polarity scores for SentimentIntensityAnalyzer
    score = sia.polarity_scores(comment)
    score['replies'] = comment
    ps.append(score)

pol_df = pd.DataFrame.from_records(ps)

#set bullish if 1, bearish if 0, neutral if 0
pol_df['sentiment'] = 0
pol_df.loc[pol_df['compound'] > 0.2, 'sentiment'] = 1
pol_df.loc[pol_df['compound'] < -0.2, 'sentiment'] = -1


bullish = sentiment_analysis(pol_df[pol_df.sentiment==1].replies)
bearish = sentiment_analysis(pol_df[pol_df.sentiment==-1].replies)
neutral = sentiment_analysis(pol_df[pol_df.sentiment==0].replies)

#bullish ticker & names 
df_bull_ticker = pd.DataFrame(nltk.FreqDist([crypto for bull in text_cleaner(bullish) for crypto in ticker if bull==crypto]).most_common(5), columns=['Name', 'Frequencies'])
df_bull_slug = pd.DataFrame(nltk.FreqDist([crypto for bull in text_cleaner(bullish) for crypto in slug if bull==crypto]).most_common(5), columns=['Name', 'Frequencies'])
#bearish ticker & names
df_bear_ticker = pd.DataFrame(nltk.FreqDist([crypto for bear in text_cleaner(bearish) for crypto in ticker if bear==crypto]).most_common(5), columns=['Name', 'Frequencies'])
df_bear_slug = pd.DataFrame(nltk.FreqDist([crypto for bear in text_cleaner(bearish) for crypto in slug if bear==crypto]).most_common(5), columns=['Name', 'Frequencies'])
#neutral ticker & names
df_neutral_ticker = pd.DataFrame(nltk.FreqDist([crypto for neu in text_cleaner(neutral) for crypto in ticker if neu==crypto]).most_common(5), columns=['Name', 'Frequencies'])
df_neutral_slug = pd.DataFrame(nltk.FreqDist([crypto for neu in text_cleaner(neutral) for crypto in slug if neu==crypto]).most_common(5), columns=['Name', 'Frequencies'])




'  \n    bulltickerfig = px.bar(df_bull_ticker, x=df_bull_ticker.Name, y=df_bull_ticker.Frequencies)\n    bullslugfig = px.bar(df_bull_slug, x=df_bull_slug.Name, y=df_bull_slug.Frequencies)\n    bulltickergraphJSON = json.dumps(bulltickerfig, cls=plotly.utils.PlotlyJSONEncoder)\n    bullsluggraphJSON = json.dumps(bullslugfig, cls=plotly.utils.PlotlyJSONEncoder)\n\n    beartickerfig = px.bar(df_bear_ticker, x=df_bear_ticker.Name, y=df_bear_ticker.Frequencies)\n    bearslugfig = px.bar(df_bear_slug, x=df_bear_slug.Name, y=df_bear_slug.Frequencies)\n    beartickergraphJSON = json.dumps(beartickerfig, cls=plotly.utils.PlotlyJSONEncoder)\n    bearsluggraphJSON = json.dumps(bearslugfig, cls=plotly.utils.PlotlyJSONEncoder)\n\n    neutickerfig = px.bar(df_neutral_ticker, x=df_neutral_ticker.Name, y=df_neutral_ticker.Frequencies)\n    neuslugfig = px.bar(df_neutral_slug, x=df_neutral_slug.Name, y=df_neutral_slug.Frequencies)\n    neutickergraphJSON = json.dumps(neutickerfig, cls=plotly.utils.Pl

In [71]:
wall['created_utc'] = wall['created_utc'].apply(lambda x: datetime.fromtimestamp(x))

In [72]:
df_bull = pd.concat([df_bull_ticker, df_bull_slug], axis=1, ignore_index=True)
df_bear = pd.concat([df_bear_ticker, df_bear_slug], axis=1, ignore_index=True)
df_neutral = pd.concat([df_neutral_ticker, df_neutral_slug], axis=1, ignore_index=True)
df_all = pd.concat([df_bull, df_bear, df_neutral], axis=1, ignore_index=True)


In [73]:
df_all.columns = ['Bull Ticker', 'Bull_T Freq','Bull Slug','Bull_S Freq','Bear Ticker', 'Bear_T Freq','Bear Slug','Bear_S Freq','Neutral Ticker', 'Neutral_T Freq','Neutral Slug','Neutral_S Freq']


In [74]:
df_all = df_all.join(wall['created_utc'])

In [75]:
df_all.set_index('created_utc', inplace=True)
wall.set_index('created_utc', inplace=True)

In [76]:
wall.head()

,title,author,id,comments,upvote_score
created_utc,,,,,
2023-01-30 10:42:58,"Daily General Discussion - January 30, 2023 (G...",alander4,j6ieu53,The market has basically been taking 2 steps f...,6
2023-01-30 10:57:24,"Daily General Discussion - January 30, 2023 (G...",Dark_Raiden_,j6ih19m,Today's scary prices are yesterday's pumps.\n\...,7
2023-01-29 19:46:46,"Daily General Discussion - January 30, 2023 (G...",marsangelo,j6fs8l2,FOMC will likely be as predicted but tech earn...,8
2023-01-30 04:46:12,"Daily General Discussion - January 30, 2023 (G...",Ghost_Lagoon,j6hdxxw,"Jeeez guys relax, BTC dipped 2%. Alts do alway...",7
2023-01-30 09:40:49,"Daily General Discussion - January 30, 2023 (G...",GenesisZor,j6i5w9j,Finally managed to set up my vault 😍,7


In [77]:
df_all.head()

,Bull Ticker,Bull_T Freq,Bull Slug,Bull_S Freq,Bear Ticker,Bear_T Freq,Bear Slug,Bear_S Freq,Neutral Ticker,Neutral_T Freq,Neutral Slug,Neutral_S Freq
created_utc,,,,,,,,,,,,
2023-01-30 10:42:58,one,198,bitcoin,115,one,81,bitcoin,53,one,86,bitcoin,48
2023-01-30 10:57:24,btc,105,polkadot,17,eth,44,chain,8,btc,77,monero,12
2023-01-29 19:46:46,eth,80,ethereum,16,btc,41,ethereum,7,eth,37,polygon,7
2023-01-30 04:46:12,op,52,chain,13,ever,16,cardano,5,matic,18,bnb,6
2023-01-30 09:40:49,bit,32,polygon,9,etc,14,monero,4,op,18,chain,5


In [78]:
wall.dtypes

title           object
author          object
id              object
comments        object
upvote_score     int64
dtype: object

In [79]:
wall.title = wall.title.astype("string")
wall.author = wall.author.astype("string")
wall.id = wall.id.astype("string")
wall.comments = wall.comments.astype("string")

In [81]:
postgres_login('','','',6969,'reddit','reddit_r_Cryptocurrency')

In [82]:
query = """SELECT comments, author
FROM public."reddit_r_Cryptocurrency"
WHERE upvote_score >10"""

pd.read_sql(query, con=engine)

,comments,author
0,Need a tl;dr for the tl;dr? \n\nOn-chain analy...,002timmy
1,forget all the excuses. If Celcius and 3AC wo...,moldyjellybean
2,This FTX/Alameda fiasco is like peeling an oni...,Kappatalizable
3,Crypto will be a better space when major marke...,DoubleFaulty1
4,I know nothing about this drama but the most i...,big13lackliz4rd
...,...,...
254,be prepared to lose all your money,Killertimme
255,they have pegged there stable coin to a blunt ...,SufficientNet9227
256,"Unfortunately, the increased hashrate does not...",Olmops
257,I can see them going for the wrong regulations...,Intelligent_Page2732


In [83]:
query = """SELECT *
FROM public."reddit_r_Cryptocurrency"
LIMIT 10"""

pd.read_sql(query, con=engine)

,created_utc,title,author,id,comments,upvote_score
0,2023-01-30 10:42:58,"Daily General Discussion - January 30, 2023 (G...",alander4,j6ieu53,The market has basically been taking 2 steps f...,6
1,2023-01-30 10:57:24,"Daily General Discussion - January 30, 2023 (G...",Dark_Raiden_,j6ih19m,Today's scary prices are yesterday's pumps.\n\...,7
2,2023-01-29 19:46:46,"Daily General Discussion - January 30, 2023 (G...",marsangelo,j6fs8l2,FOMC will likely be as predicted but tech earn...,8
3,2023-01-30 04:46:12,"Daily General Discussion - January 30, 2023 (G...",Ghost_Lagoon,j6hdxxw,"Jeeez guys relax, BTC dipped 2%. Alts do alway...",7
4,2023-01-30 09:40:49,"Daily General Discussion - January 30, 2023 (G...",GenesisZor,j6i5w9j,Finally managed to set up my vault 😍,7
5,2023-01-30 10:06:51,"Daily General Discussion - January 30, 2023 (G...",cryotosensei,j6i9ivi,No other coin had the cachet of Bitcoin\n\nJus...,7
6,2023-01-29 19:02:53,"Daily General Discussion - January 30, 2023 (G...",IAmNocturneAMA,j6flpx0,"New daily, new me - we run it back again",3
7,2023-01-29 21:03:25,"Daily General Discussion - January 30, 2023 (G...",milonuttigrain,j6g33cg,"As of today, the market cap of Bitcoin (BTC) i...",6
8,2023-01-30 10:43:14,"Daily General Discussion - January 30, 2023 (G...",C-Dawg420,j6ievmi,Why do people dislike Charles Hoskinson?,5
9,2023-01-30 10:50:03,"Daily General Discussion - January 30, 2023 (G...",MK2809,j6ifwo6,Just typed 'mass adoption' into Google and the...,5


In [84]:
quer = """SELECT comments, upvote_score
FROM public."reddit_r_Cryptocurrency"
WHERE comments LIKE '%%bitcoin%%'
OR comments LIKE '%%ethereum%%'
ORDER BY upvote_score DESC
"""

pd.read_sql(quer, con=engine)

,comments,upvote_score
0,tldr; Bitcoin reached a fresh five-month high ...,33
1,He must be new. All diehard bitcoiners have lo...,9
2,The guy wasn’t investing in bitcoin. Buying 10...,9
3,How does it not make sense? Fiat is devaluing...,7
4,Yeah I've made the mistake of explaining bitco...,7
...,...,...
63,Never ever talk about crypto with people even ...,1
64,I'm on crypto and bitcoin since 2021. And noti...,0
65,"Finally 1M in Q12 2021!\n\nJokes aside, bitcoi...",0
66,Just like in the last 5 years where it failed ...,0
